<a href="https://colab.research.google.com/github/rahiakela/introduction-to-time-series-forecasting-with-python/blob/part-1-data-preparation/2_basic_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Feature Engineering

Time Series data must be re-framed as a supervised learning dataset before we can start using machine learning algorithms. There is no concept of input and output features in time series.

Instead, we must choose the variable to be predicted and use feature engineering to construct all of the inputs that will be used to make predictions for future time steps.

## Feature Engineering for Time Series

A time series dataset must be transformed to be modeled as a supervised learning problem.

That is something that looks like:

```python
time 1, value 1
time 2, value 2
time 3, value 3
```

To something that looks like:

```python
input 1, output 1
input 2, output 2
input 3, output 3
```

So that we can train a supervised learning algorithm. Input variables are also called features in the field of machine learning, and the task before us is to create or invent new input features from our time series dataset. Ideally, we only want input features that best help the learning methods model the relationship between the inputs (X) and the outputs (y) that we would like
to predict.

We will look at three classes of features that we can create from our
time series dataset:

* **Date Time Features**: these are components of the time step itself for each observation.
* **Lag Features**: these are values at prior time steps.
* **Window Features**: these are a summary of values over a fixed window of prior time steps.

## Goal of Feature Engineering

The goal of feature engineering is to provide strong and ideally simple relationships between new input features and the output feature for the supervised learning algorithm to model. In effect, we are moving complexity.

Complexity exists in the relationships between the input and output data. In the case of time series, there is no concept of input and output variables; we must invent these too and frame the supervised learning problem from scratch.

The difficulty is that we do not know the underlying inherent functional relationship between inputs and outputs that we're trying to expose. If we did know, we probably would not need machine learning. Instead, the only feedback we have is the performance of models developed on the supervised learning datasets or views of the problem we create. 

In effect, the best default strategy is to use all the knowledge available to create many good datasets from your time series dataset and use model performance (and other project requirements) to help determine what good features and good views of your problem happen to be.

For clarity, we will focus on a univariate (one variable) time series dataset in the examples, but these methods are just as applicable to multivariate time series problems.

## Minimum Daily Temperatures Dataset

This dataset describes the minimum daily temperatures over 10 years (1981-1990) in the city Melbourne, Australia. The units are in degrees Celsius and there are 3650 observations. 

In [7]:
import pandas as pd
import numpy as np
from pandas import Series
from pandas import DataFrame

series = pd.read_csv('daily-minimum-temperatures-in-me.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
series.head()

Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Name: Daily minimum temperatures in Melbourne, Australia, 1981-1990, dtype: object

In [8]:
dataframe = DataFrame()
dataframe['month'] = [series.index[i].month for i in range(len(series))]
dataframe['day'] = [series.index[i].day for i in range(len(series))]
dataframe['temperature'] = [series.index[i] for i in range(len(series))]
print(dataframe.head(5))

   month  day temperature
0      1    1  1981-01-01
1      1    2  1981-01-02
2      1    3  1981-01-03
3      1    4  1981-01-04
4      1    5  1981-01-05


Using just the month and day information alone to predict temperature is not sophisticated and will likely result in a poor model. Nevertheless, this information coupled with additional engineered features may ultimately result in a better model. You may enumerate all the properties of a time-stamp and consider what might be useful for your problem, such as:

* Minutes elapsed for the day.
* Hour of day.
* Business hours or not.
* Weekend or not.
* Season of the year.
* Business quarter of the year.
* Daylight savings or not.
* Public holiday or not.
* Leap year or not.

You can use binary ag features as well, like whether or not the observation was recorded on a public holiday. In the case of the minimum temperature dataset, maybe the season would be more relevant. It is creating domain-specific features like this that are more likely to add value to
your model. Date-time based features are a good start, but it is often a lot more useful to include the values at previous time steps.

## Lag Features

Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems. The simplest approach is to predict the value at the next time (t+1) given the value at the current time (t).

```python
Value(t), Value(t+1)
Value(t), Value(t+1)
Value(t), Value(t+1)
```

The Pandas library provides the shift() function1 to help create these shifted or lag features from a time series dataset. Shifting the dataset by 1 creates the t column, adding a NaN (unknown) value for the first row. The time series dataset without a shift represents the t+1.

Let's make this concrete with an example. The first 3 values of the temperature dataset are 20.7, 17.9, and 18.8. The shifted and unshifted lists of temperatures for the first 3 observations are therefore:

```python
Shifted, Original
NaN,     20.7
20.7,    17.9
17.9,    18.8
```

We can concatenate the shifted columns together into a new DataFrame using the concat() function along the column axis (axis=1). 

Putting this all together, below is an example of creating a lag feature for our daily temperature dataset. The values are extracted from the
loaded series and a shifted and unshifted list of these values is created.



In [9]:
temps = DataFrame(series.values)
dataframe = pd.concat([temps.shift(1), temps], axis=1)
dataframe.columns = ['t', 't+1']
print(dataframe.head())

      t   t+1
0   NaN  20.7
1  20.7  17.9
2  17.9  18.8
3  18.8  14.6
4  14.6  15.8


You can see that we would have to discard the first row to use the dataset to train a supervised learning model, as it does not contain enough data to work with. The addition of lag features is called the sliding window method, in this case with a window width of 1. It is as though we are sliding our focus along the time series for each observation with an interest in only what is within the window width. We can expand the window width and include more lagged features.

For example, below is the above case modified to include the last 3 observed
values to predict the value at the next time step.

In [10]:
# create lag features
temps = DataFrame(series.values)
dataframe = pd.concat([temps.shift(3), temps.shift(2), temps.shift(1), temps], axis=1)
dataframe.columns = ['t-2', 't-1', 't', 't+1']
print(dataframe.head())

    t-2   t-1     t   t+1
0   NaN   NaN   NaN  20.7
1   NaN   NaN  20.7  17.9
2   NaN  20.7  17.9  18.8
3  20.7  17.9  18.8  14.6
4  17.9  18.8  14.6  15.8


Again, you can see that we must discard the first few rows that do not have enough data to train a supervised model. A dificulty with the sliding window approach is how large to make the window for your problem. 

Perhaps a good starting point is to perform a sensitivity analysis and try a suite of different window widths to in turn create a suite of different views of your dataset and see which results in better performing models. There will be a point of diminishing returns.

Perhaps you need a lag value from last week, last month, and last year. Again, this comes down to the specific domain. In the case of the
temperature dataset, a lag value from the same day in the previous year or previous few years may be useful. We can do more with a window than include the raw values.

## Rolling Window Statistics

A step beyond adding raw lagged values is to add a summary of the values at previous time steps. We can calculate summary statistics across the values in the sliding window and include these as features in our dataset. Perhaps the most useful is the mean of the previous few values, also called the rolling mean.

We can calculate the mean of the current and previous values and use that to predict the next value. For the temperature data, we would have to wait 3 time steps before we had 2 values to take the average of before we could use that value to predict a 3rd value.

```python
mean(t-1, t), t+1
mean(20.7, 17.9), 18.8
19.3, 18.8
```

Pandas provides a rolling() function3 that creates a new data structure with the window of values at each time step. We can then perform statistical functions on the window of values collected for each time step, such as calculating the mean. First, the series must be shifted.
Then the rolling dataset can be created and the mean values calculated on each window of two values. 

Here are the values in the first three rolling windows:

```python
#, Window Values
1, NaN
2, NaN, 20.7
3, 20.7, 17.9
```

This suggests that we will not have usable data until the 3rd row. Finally, as in the previous section, we can use the concat() function to construct a new dataset with just our new columns.

The example below demonstrates how to do this with Pandas with a window size of 2.

In [0]:
series = pd.read_csv('daily-minimum-temperatures-in-me.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
temps = DataFrame(series.values)
shifted = temps.shift(1)
window = shifted.rolling(window=2)
means = window.mean().astype(float)
dataframe = pd.concat([means, temps], axis=1)
dataframe.columns = ['mean(t-1, t)', 't+1']
print(dataframe.head())

In [32]:
series = pd.read_csv('daily-minimum-temperatures-in-me.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
temps = DataFrame(series.values)
window = temps.expanding()
dataframe = pd.concat([window.min(), window.mean(), window.max(), temps.shift(-1)], axis=1)
dataframe.columns = ['min', 'mean', 'max', 't+1']
print(dataframe.head(5))


DataError: ignored